In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing_extensions import List, TypedDict
from langchain_core.documents import Document
from langchain import hub
from langgraph.graph import START, StateGraph, MessagesState



load_dotenv()

openrouter_key = os.getenv("OPENROUTER_API_KEY")
openrouter_base_url = os.getenv("OPENAI_BASE_URL")

In [ ]:
print(openrouter_key)

In [ ]:
llm = ChatOpenAI(
    model_name="deepseek/deepseek-chat-v3.1:free",
    api_key=openrouter_key,
    base_url=openrouter_base_url
)


emb = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    encode_kwargs={"normalize_embeddings": True}  # normalize for cosine similarity
)


vector_store = InMemoryVectorStore(emb)

In [ ]:
print(llm.invoke("Hello, LangGraph + OpenRouter!").content)

In [ ]:
pdf_path = r"C:\Users\user\Desktop\Khawaja-Bilal-Ahmad.pdf"
loader = PyPDFLoader(pdf_path)
docs = loader.load()
print(f"Loaded {len(docs)} pages")
print(docs[0].page_content[:300])

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

_ = vector_store.add_documents(documents=all_splits)

In [9]:
prompt = hub.pull("rlm/rag-prompt")

class State(TypedDict):
    question: str
    context: List[Document]
    answer: str



def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return{"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


In [ ]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()



In [ ]:
response = graph.invoke({"question": "provide me his skills?"})
print(response["answer"])

In [ ]:
print(f"Context: {response['context']}\n\n")
print(f"Answer: {response['answer']}")